In [0]:
# This notebook is meant to run batch inference on the top of new iris samples

In [0]:
%pip install --quiet mlflow==2.22.0 databricks-feature-engineering==0.12.1

In [0]:
dbutils.library.restartPython() 

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient
from sklearn import datasets
from mlflow.pyfunc import load_model
import pandas as pd
import mlflow

In [0]:
dbutils.widgets.text("catalog_name", "pedroz_e2edata_dev")
catalog_name = dbutils.widgets.get("catalog_name")

In [0]:
model_name = f'{catalog_name}.default.iris_model'

In [0]:
# Pull the dataset for running the inference
iris_samples = datasets.load_iris(as_frame=True)
df_samples = pd.DataFrame(data = iris_samples['data'], columns = iris_samples['feature_names'])
df_samples.columns = df_samples.columns.str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df_samples.head()

In [0]:
model_uri = f"models:/{model_name}@champion"
model = load_model(model_uri)

In [0]:
predictions = model.predict(df_samples)
df_samples['prediction'] = predictions

df_samples.head()

In [0]:
df_spark = spark.createDataFrame(df_samples)
df_spark.write.mode("overwrite").saveAsTable(f"{catalog_name}.default.iris_inferences")